In [4]:
KAFKA_BOOTSTRAP = "ec2-3-8-99-245.eu-west-2.compute.amazonaws.com:9092"
TOPIC = "stocks_demo"
CSV_PATH = r"C:\Users\Administrator\OneDrive - KOROGLU AGATHA\DE\kafka_streaming\kafka_project\indexProcessed.csv"
SEND_INTERVAL_SEC = 1.0   # 1 message per second (change to taste)


In [ ]:
import pandas as pd, json, time, random
from kafka import KafkaProducer

# Load once, sample rows forever
df = pd.read_csv(CSV_PATH)

producer = KafkaProducer(
    bootstrap_servers=[KAFKA_BOOTSTRAP],
    acks="all",
    linger_ms=0,  # send immediately (lowest latency)
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

try:
    i = 0
    while True:
        rec = df.iloc[random.randrange(len(df))].to_dict()
        # Optional latency measurement:
        # rec["_sent_ts_ns"] = time.time_ns()
        producer.send(TOPIC, rec)
        # producer.flush()  # not needed with linger_ms=0; uncomment if you want hard flush per msg
        i += 1
        if i % 100 == 0:
            print(f"sent {i} messages…")
        time.sleep(SEND_INTERVAL_SEC)
finally:
    producer.flush()
    producer.close()
